Import Packages

In [84]:
import sys
sys.path.append('C:/Users/Josh Ellis/OneDrive - University of Nebraska at Omaha/COURSES/FALL_2022/ISQA8156-820/course-project')

import pandas as pd
import numpy as np
import scipy.stats as stats
import plotly.express as px
from statsmodels.formula.api import ols
import statsmodels.api as sm

from src.stat_methods import FishersLSD
from src.processing import prepost_transform

### Prepare Data

Read pre-processed data from github
- See src/processing/raw_data_processing.py for raw data processing steps

In [80]:
data = pd.read_csv('https://raw.githubusercontent.com/josh97ellis/omaha-girls-rock-analysis/master/data/processed/girls_rock_data.csv')

print(f'shape: {data.shape}')
data.head()

shape: (360, 22)


,client,age_group,year,test_type,age,years_at_camp,race/ethnicity,zip_code,15. Wear the kind of clothes you like even if they are different from what others wear.,"16. In a line-up, tell a student who pushes in front of you to wait his or her turn.",...,19. Join a school club or sports team.,20. Express your feelings to another kid.,21. Ask someone over to your house on a Saturday.,22. Ask someone to go to a school dance or movie with you.,23. Go to a party where you are sure you won’t know any of the kids.,24. Ask another student for help when you need it.,25. Make friends with kids your age.,"1. You can learn new things, but you can’t really change your basic intelligence.",2. Your intelligence is something about you that you can’t change very much.,3. You have a certain amount of intelligence and you really can’t do much to change it.
0,1681042018,older group,2018,pre-test,14,2,caucasian,68104.0,4,4.0,...,4.0,4,5,4.0,4.0,4.0,4.0,5.0,5.0,5.0
1,2681042018,older group,2018,pre-test,14,1,caucasian,68104.0,6,5.0,...,7.0,5,5,5.0,6.0,7.0,6.0,3.0,3.0,3.0
2,3515342018,older group,2018,pre-test,16,5,caucasian,51534.0,7,4.0,...,5.0,5,4,5.0,4.0,6.0,6.0,3.0,4.0,5.0
3,4685062018,older group,2018,pre-test,15,5,caucasian,68506.0,5,2.0,...,1.0,2,2,3.0,2.0,2.0,3.0,5.0,6.0,6.0
4,5681052018,older group,2018,pre-test,13,4,caucasian,68105.0,7,4.0,...,7.0,7,7,7.0,7.0,7.0,7.0,2.0,3.0,3.0


Transform Data to be analyzed
- See src/processing/prepost_transform.py for transformation processing

In [79]:
data_transformed = prepost_transform(data)

print(f'shape: {data_transformed.shape}')
data_transformed.head()

shape: (2520, 9)


,client,age_group,year,age,race/ethnicity,question,score_pretest,score_posttest,delta
0,1681042018,older group,2018,14,caucasian,15,4.0,7.0,3.0
1,2681042018,older group,2018,14,caucasian,15,6.0,5.0,-1.0
2,3515342018,older group,2018,16,caucasian,15,7.0,7.0,0.0
3,4685062018,older group,2018,15,caucasian,15,5.0,5.0,0.0
4,5681052018,older group,2018,13,caucasian,15,7.0,7.0,0.0


---

### Research Topic 1
Are there significant differences in answer ratings for various outcomes between the younger group and the older group? This topic will help OGR better understand which survey questions significantly improve over throughout the duration of the program.

**Question 1:**
- Which questions by age group show a significant improvement in survey scores after participating in the Omaha Girls Rock Program?

- **Statistical Test**:
    - Right-tailed dependent (matched-sample) t-test at a 5% level of significance
    - Individual test for each question-age group combination (2 groups x 14 questions)

- **Hypotheses**:
    - $H_0: \mu_d \le 0$
    - $H_a: \mu_d > 0$

In [95]:
age_data = data_transformed[['age_group', 'question', 'score_pretest', 'score_posttest', 'delta']].copy()

# Prepare a list of individual dataframes for each test (28 Total)
sample_data_list = []
for age_group in age_data['age_group'].unique():
    for question in age_data['question'].unique():
        test_data = (
            age_data[
                (age_data['age_group'] == age_group) &
                (age_data['question'] == question)]
            [['age_group', 'question', 'score_pretest', 'score_posttest']]
        )
        sample_data_list.append(test_data)

# field lists for results table
group_li = []
question_li = []
pretest_mean_li = []
posttest_mean_li = []
test_statistic_li = []
pvalue_li = []
conclusion_li = []

# Run a dependent t-test for each dataframe in data_list and store results in the respective field list
for sample in sample_data_list:
    # Get Pre and Post Test Scores
    pretest_scores = sample['score_pretest']
    posttest_scores = sample['score_posttest']

    # Run dependent t-test
    test = stats.ttest_rel(
        posttest_scores,
        pretest_scores,
        alternative='greater')
    
    # Create Results
    pvalue = test[1]
    
    if round(pvalue, 3) <= .050:
        conclusion_li.append('Significant')
    else:
        conclusion_li.append('Not Significant')
    
    pvalue_li.append(round(pvalue, 3))
    test_statistic_li.append(round(test[0], 3))
    group_li.append(sample.iloc[0,0])
    question_li.append(sample.iloc[0,1])
    pretest_mean_li.append(round(pretest_scores.mean(), 3))
    posttest_mean_li.append(round(posttest_scores.mean(), 3))


# create results table
results = pd.DataFrame({
    'group': group_li,
    'question': question_li,
    'pre-test mean': pretest_mean_li,
    'post-test mean': posttest_mean_li,
    'test statistic': test_statistic_li,
    'p-value': pvalue_li,
    'conclusion': conclusion_li
})

results

,group,question,pre-test mean,post-test mean,test statistic,p-value,conclusion
0,older group,15,5.814,6.012,1.660,0.050,Significant
1,older group,16,5.244,5.616,2.833,0.003,Significant
2,older group,17,5.616,5.779,1.177,0.121,Not Significant
3,older group,18,5.791,5.988,1.805,0.037,Significant
4,older group,19,5.907,6.081,1.706,0.046,Significant
5,older group,20,4.698,5.198,3.209,0.001,Significant
6,older group,21,5.953,5.907,-0.376,0.646,Not Significant
7,older group,22,4.640,4.977,2.154,0.017,Significant
8,older group,23,3.581,4.221,4.124,0.000,Significant
9,older group,24,5.012,5.535,3.565,0.000,Significant


**Question 2:**
- Is there a statistically significant difference between the delta scores of the Older Group and Younger Group (per question)

- **Statistical Test**:
    - Independent t-test about the means between the delta scores for the Younger Group ($\mu_1$) and Older Group ($\mu_2$).
    - Conduct an individual t-test for each question-age group combination (2 groups x 14 questions)

- **Hypotheses:**
    - $𝐻_0: \mu_1 − \mu_2 \le 0$
    - $𝐻_a: \mu_1 − \mu_2 > 0$


In [68]:
mean_younger_li = []
mean_older_li = []
question_li = []
pvalue_li = []
test_statistic_li = []
conclusion_li = []

for question in age_data['question'].unique():
    younger_delta = age_data[(age_data['question']==question) & (age_data['age_group']=='younger group')]['delta']
    older_delta = age_data[(age_data['question']==question) & (age_data['age_group']=='older group')]['delta']
    
    ttest = stats.ttest_ind(younger_delta, older_delta)
    test_statistic = ttest[0]
    pvalue = ttest[1]
    
    if pvalue < .05:
        conclusion_li.append('Significant')
    else:
        conclusion_li.append('Not Significant')
    
    question_li.append(question)
    mean_younger_li.append(np.mean(younger_delta))
    mean_older_li.append(np.mean(older_delta))
    test_statistic_li.append(test_statistic)
    pvalue_li.append(pvalue)
    
results = pd.DataFrame({
    'question': question_li,
    'younger group mean': mean_younger_li,
    'older group mean': mean_older_li,
    'test statistic': test_statistic_li,
    'p-value': pvalue_li,
    'conclusion': conclusion_li
})

results

,question,younger group mean,older group mean,test statistic,p-value,conclusion
0,15,-0.010638,0.197674,-1.581888,0.115450,Not Significant
1,16,0.159574,0.372093,-1.256467,0.210593,Not Significant
2,17,0.031915,0.162791,-0.870677,0.385103,Not Significant
3,18,0.117021,0.197674,-0.560794,0.575643,Not Significant
4,19,0.010638,0.174419,-1.159955,0.247621,Not Significant
5,20,0.053191,0.500000,-2.336915,0.020556,Significant
6,21,0.202128,-0.046512,1.438076,0.152168,Not Significant
7,22,0.170213,0.337209,-0.793027,0.428818,Not Significant
8,23,0.329787,0.639535,-1.566386,0.119034,Not Significant
9,24,0.127660,0.523256,-2.137105,0.033955,Significant


---

### Research Topic 2
Does the race/ethnicity of participants influence the change in scores for various questions over time? 

**Experiment 1:** One-way ANOVA

In [86]:
race_data = data_transformed[['race/ethnicity', 'question', 'score_pretest', 'score_posttest', 'delta']].copy()
race_data.rename(columns={'race/ethnicity': 'race'}, inplace=True)

# Lists to capture data from each iteration
question_col = []
pvalue_col = []
result_col = []

# Iterate over each survey question and perform one-way anova
for question in race_data['question'].unique():    
    anova_race = race_data[race_data['question']==question]

    anova_race = anova_race[['race', 'delta']]

    model = ols('delta ~ C(race)', data=anova_race).fit()
    anova_table = sm.stats.anova_lm(model,typ=1)
    pvalue = round(anova_table.iloc[0,-1],3)

    question_col.append(question)
    pvalue_col.append(pvalue)

    if pvalue > 0.05:
        result_col.append('Not Significant')
    else:
        result_col.append('Significant')

# Create results table
results_df = pd.DataFrame({
    'question': question_col,
    'pvalue': pvalue_col,
    'result': result_col
})

results_df

,question,pvalue,result
0,15,0.985,Not Significant
1,16,0.434,Not Significant
2,17,0.584,Not Significant
3,18,0.075,Not Significant
4,19,0.804,Not Significant
5,20,0.349,Not Significant
6,21,0.106,Not Significant
7,22,0.170,Not Significant
8,23,0.129,Not Significant
9,24,0.356,Not Significant


In [9]:
px.bar(
       results_df, x='question', y='pvalue',
       color='result',
       title='One-Way Anova Results <br><i>Treatment: race/ethnicity | Response: Change between pre-test and post-test Scores<br><i>Level of Significance = 5%',
       text_auto=True, template='simple_white')

**Experiment 2:** Two-way ANOVA (Factorial Design)
- Factor A: Race/Ethnicity (4 Levels)
- Factor B: Question (14 Levels)

In [38]:
race_data = data_transformed[['race/ethnicity', 'question', 'delta']].copy()
race_data.columns = ['race', 'question', 'delta']

model = ols('delta ~ C(race, Sum)*C(question, Sum)', data=race_data).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
"C(race, Sum)",18.226502,3.0,5.080615,0.001655
"C(question, Sum)",46.627381,13.0,2.999383,0.000217
"C(race, Sum):C(question, Sum)",43.979047,39.0,0.943008,0.571615
Residual,2946.500007,2464.0,NaN,NaN


- Race: The mean change in scores from pre-test to post-test differs by race
- question: The mean change in scores from pre-test to post-test differs by race
- Interaction: The interaction between race and question is not significant

In [94]:
px.box(race_data, x='race', y='score_posttest')

In [90]:
race_data

,race,question,score_pretest,score_posttest,delta
0,caucasian,15,4.0,7.0,3.0
1,caucasian,15,6.0,5.0,-1.0
2,caucasian,15,7.0,7.0,0.0
3,caucasian,15,5.0,5.0,0.0
4,caucasian,15,7.0,7.0,0.0
...,...,...,...,...,...
2515,caucasian,3,4.0,5.0,1.0
2516,multi-racial,3,4.0,4.0,0.0
2517,caucasian,3,6.0,6.0,0.0
2518,caucasian,3,3.0,4.0,1.0
